In [ ]:
SYFT_VERSION = ">=0.9,<1.0.0"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [ ]:
# stdlib
import json

# syft absolute
import syft as sy
from syft import autocache

sy.requires(SYFT_VERSION)

In [ ]:
server = sy.orchestra.launch(
    name="test-datasite-1",
    dev_mode=True,
    reset=True,
    create_producer=True,
)

In [ ]:
datasite_client = server.login(email="info@openmined.org", password="changethis")

# Uploading a Blobfile from multiple local files

In [ ]:
a_dataset_url = (
    "https://raw.githubusercontent.com/OpenMined/datasets/main/testdatasyft/a.json"
)
a_file = autocache(a_dataset_url, "json")
a_file

In [ ]:
b_dataset_url = (
    "https://raw.githubusercontent.com/OpenMined/datasets/main/testdatasyft/b.json"
)
b_file = autocache(b_dataset_url, "json")
b_file

In [ ]:
data_ptr = datasite_client.upload_files([a_file, b_file])

In [ ]:
data_ptr[0].read()

# Iterating over lines of a file

In [ ]:
x_dataset_url = (
    "https://raw.githubusercontent.com/OpenMined/datasets/main/testdatasyft/lines.jsonl"
)
x_file = autocache(x_dataset_url, "jsonl")
x_file

In [ ]:
lines_file_pptr = datasite_client.upload_files(x_file)[0].syft_action_data

In [ ]:
res = []
for y in lines_file_pptr.iter_lines():
    d = json.loads(y)
    print(d)
    res.append(d["x"])
assert sum(res) == 3

# Creating a dataset with a list of files as Asset

In [ ]:
ds = sy.Dataset(
    name="my dataset",
    asset_list=[sy.Asset(name="data", data=data_ptr, mock=sy.ActionObject.empty())],
)

In [ ]:
datasite_client.upload_dataset(ds)

# Running a syft_function on the files

In [ ]:
@sy.syft_function_single_use(files=data_ptr)
def sum_trade_value_mil(files):
    # stdlib
    import json

    a_file, b_file = files
    data_1 = json.loads(a_file.read())
    data_2 = json.loads(b_file.read())
    res = data_1["x"] + data_2["x"]
    return res

In [ ]:
assert sum_trade_value_mil(files=data_ptr).syft_action_data == 3

# From a single file

In [ ]:
single_data_ptr = datasite_client.upload_files(a_file)
single_data_ptr

# From a folder

In [ ]:
if False:
    datasite_client.upload_files("./path/to/folder")

#### Cleanup local datasite server

In [ ]:
if server.server_type.value == "python":
    server.land()